In [1]:
console.log("hello");

hello


In [2]:
const tslab = require('tslab');
console.log('Hello, tslab!')
console.log('Versions:', tslab.versions)

Hello, tslab!
Versions: { tslab: '1.0.15', typescript: '4.1.2', node: 'v16.2.0' }


In [3]:
const matrix = require('node-sense-hat').Leds;

In [4]:
// To fill with a single color instead
//matrix.clear([0, 0, 0]);
matrix.clear([0, 255, 0]);

In [ ]:
matrix.clear([0, 0, 0]);

In [5]:
const imu = require("node-sense-hat").Imu;

const IMU = new imu.IMU();

function displayValue(err, data) {
  if (err !== null) {
    console.error("Could not read sensor data: ", err);
    return;
  }

  console.log("Accelleration is: ", JSON.stringify(data.accel, null, "  "));
  console.log("Gyroscope is: ", JSON.stringify(data.gyro, null, "  "));
  console.log("Compass is: ", JSON.stringify(data.compass, null, "  "));
  console.log("Fusion data is: ", JSON.stringify(data.fusionPose, null, "  "));
  

  console.log("Temp is: ", data.temperature);
  console.log("Pressure is: ", data.pressure);
  console.log("Humidity is: ", data.humidity);
  
};
IMU.getValue(displayValue);

Accelleration is:  {
  "x": -0.011711999773979187,
  "y": 0.1320039927959442,
  "z": 1.3149160146713257
}
Gyroscope is:  {
  "x": -0.1713477075099945,
  "y": 0.05100724846124649,
  "z": 0.04459316283464432
}
Compass is:  {
  "x": 2.9204001426696777,
  "y": 5.608400344848633,
  "z": 11.03760051727295
}
Fusion data is:  {
  "x": 0.10005445778369904,
  "y": 0.008862254209816456,
  "z": -0.9770402908325195
}
Temp is:  24.692646026611328
Pressure is:  969.5498046875
Humidity is:  69.37545776367188


In [6]:
IMU.getValue(displayValue);

Accelleration is:  {
  "x": -0.008539999835193157,
  "y": 0.13102799654006958,
  "z": 0.9781959652900696
}
Gyroscope is:  {
  "x": 0.006108652334660292,
  "y": 0.005192354787141085,
  "z": 0.02474004216492176
}
Compass is:  {
  "x": 5.088720321655273,
  "y": 9.899120330810547,
  "z": 19.56808090209961
}
Fusion data is:  {
  "x": 0.1045319214463234,
  "y": 0.01045367494225502,
  "z": -0.9611996412277222
}
Temp is:  24.657352447509766
Pressure is:  969.60595703125
Humidity is:  70.21825408935547


In [7]:
interface Entry {
   timestamp: Date;
   value: number;
}

var points = Array<Entry>();

In [8]:


function addToArray(err, data) {
  if (err !== null) {
    console.error("Could not read sensor data: ", err);
    return;
  }
    
    var coeff = 1000 * 60 * 1;
    var date = new Date();  //or use any other date
    var rounded = new Date(Math.round(date.getTime() / coeff) * coeff);
    let entry = { timestamp: rounded, value: data.temperature } as Entry;
    points.push(entry);
};

In [9]:
IMU.getValue(addToArray);

In [24]:
points

[
  { timestamp: 2022-04-07T01:07:00.000Z, value: 26.192646026611328 },
  { timestamp: 2022-04-07T01:08:00.000Z, value: 27.05735206604004 },
  { timestamp: 2022-04-07T01:09:00.000Z, value: 27.816177368164062 },
  { timestamp: 2022-04-07T01:10:00.000Z, value: 28.398529052734375 },
  { timestamp: 2022-04-07T01:11:00.000Z, value: 28.804410934448242 },
  { timestamp: 2022-04-07T01:12:00.000Z, value: 29.351470947265625 },
  { timestamp: 2022-04-07T01:13:00.000Z, value: 29.598529815673828 },
  { timestamp: 2022-04-07T01:14:00.000Z, value: 29.84558868408203 },
  { timestamp: 2022-04-07T01:15:00.000Z, value: 30.233823776245117 },
  { timestamp: 2022-04-07T01:16:00.000Z, value: 30.48088264465332 },
  { timestamp: 2022-04-07T01:17:00.000Z, value: 30.674999237060547 },
  { timestamp: 2022-04-07T01:18:00.000Z, value: 30.851470947265625 },
  { timestamp: 2022-04-07T01:19:00.000Z, value: 31.01029396057129 },
  { timestamp: 2022-04-07T01:20:00.000Z, value: 31.186763763427734 }
]


In [11]:
points = []

[]


In [12]:
let timerID = setInterval(function() {
    IMU.getValue(addToArray);
}, 60 * 1000); 


In [13]:
let timeout = setTimeout(() => { clearInterval(timerID); console.log(points); }, 15 * 60 * 1000);
 // The setInterval it cleared and doesn't run anymore.

In [14]:
//clearInterval(timerID);

In [31]:
points

[
  { timestamp: 2022-04-07T01:07:00.000Z, value: 26.192646026611328 },
  { timestamp: 2022-04-07T01:08:00.000Z, value: 27.05735206604004 },
  { timestamp: 2022-04-07T01:09:00.000Z, value: 27.816177368164062 },
  { timestamp: 2022-04-07T01:10:00.000Z, value: 28.398529052734375 },
  { timestamp: 2022-04-07T01:11:00.000Z, value: 28.804410934448242 },
  { timestamp: 2022-04-07T01:12:00.000Z, value: 29.351470947265625 },
  { timestamp: 2022-04-07T01:13:00.000Z, value: 29.598529815673828 },
  { timestamp: 2022-04-07T01:14:00.000Z, value: 29.84558868408203 },
  { timestamp: 2022-04-07T01:15:00.000Z, value: 30.233823776245117 },
  { timestamp: 2022-04-07T01:16:00.000Z, value: 30.48088264465332 },
  { timestamp: 2022-04-07T01:17:00.000Z, value: 30.674999237060547 },
  { timestamp: 2022-04-07T01:18:00.000Z, value: 30.851470947265625 },
  { timestamp: 2022-04-07T01:19:00.000Z, value: 31.01029396057129 },
  { timestamp: 2022-04-07T01:20:00.000Z, value: 31.186763763427734 },
  { timestamp: 2022-04

In [32]:
/*points = [
  { timestamp: new Date('2022-04-07T01:07:00.000Z'), value: 26.192646026611328 },
  { timestamp: new Date('2022-04-07T01:08:00.000Z'), value: 27.05735206604004 },
  { timestamp: new Date('2022-04-07T01:09:00.000Z'), value: 27.816177368164062 },
  { timestamp: new Date('2022-04-07T01:10:00.000Z'), value: 28.398529052734375 },
  { timestamp: new Date('2022-04-07T01:11:00.000Z'), value: 28.804410934448242 },
  { timestamp: new Date('2022-04-07T01:12:00.000Z'), value: 29.351470947265625 },
  { timestamp: new Date('2022-04-07T01:13:00.000Z'), value: 29.598529815673828 },
  { timestamp: new Date('2022-04-07T01:14:00.000Z'), value: 29.84558868408203 },
  { timestamp: new Date('2022-04-07T01:15:00.000Z'), value: 30.233823776245117 },
  { timestamp: new Date('2022-04-07T01:16:00.000Z'), value: 30.48088264465332 },
  { timestamp: new Date('2022-04-07T01:17:00.000Z'), value: 30.674999237060547 },
  { timestamp: new Date('2022-04-07T01:18:00.000Z'), value: 30.851470947265625 },
  { timestamp: new Date('2022-04-07T01:19:00.000Z'), value: 31.01029396057129 },
  { timestamp: new Date('2022-04-07T01:20:00.000Z'), value: 31.186763763427734 },
  { timestamp: new Date('2022-04-07T01:21:00.000Z'), value: 31.23970603942871 }
]*/

Wait for 15 minutes before continuing ....

In [33]:
const fs = require('fs');
const parse = require("csv-parse/lib/sync");
const { AnomalyDetectorClient } = require('@azure/ai-anomaly-detector');
const { AzureKeyCredential } = require('@azure/core-auth');

To create a cognitive services service and get the key
you need need to install Azure CLI and an active Azure Subscription
https://docs.microsoft.com/en-us/cli/azure/install-azure-cli

```

az group create \
    --name cognitive-services-resource-group \
    --location westus2
    
az cognitiveservices account create \
    --name anomaly-detector-resource \
    --resource-group cognitive-services-resource-group \
    --kind AnomalyDetector \
    --sku F0 \
    --location westus2 \
    --yes

az cognitiveservices account keys list \
    --name anomaly-detector-resource \
    --resource-group cognitive-services-resource-group
```

In [34]:
const dotenv = require("dotenv");
dotenv.config();
// Authentication variables
// Add your Anomaly Detector subscription key and endpoint to your environment variables.
const key = process.env["API_KEY"] || "";
const endpoint = process.env["ENDPOINT"] || "";

In [35]:
// <authentication>
let anomalyDetectorClient = new AnomalyDetectorClient(endpoint, new AzureKeyCredential(key));
// </authentication>

In [36]:
// <lastDetection>
async function lastDetection(pts) : Promise<boolean> {

    let body = { series: pts, granularity: 'minutely' }
    // Make the call to detect anomalies in the latest point of a series
    return await anomalyDetectorClient.detectLastPoint(body)
        .then((response) => {
            console.log("Latest point anomaly detection:")
            if (response.isAnomaly) {
                console.log("The latest point, in row " + points.length + ", is detected as an anomaly.")
                return true;
            } else {
                console.log("The latest point, in row " + points.length + ", is not detected as an anomaly.")
                return false;
            }
        }).catch((error) => {
            console.log(error)
            return false;
        })
}
console.log(await lastDetection(points));
// </lastDetection>

Latest point anomaly detection:
The latest point, in row 15, is not detected as an anomaly.
false


In [37]:
let newpoints = JSON.parse(JSON.stringify(points));
newpoints

[
  { timestamp: '2022-04-07T01:07:00.000Z', value: 26.192646026611328 },
  { timestamp: '2022-04-07T01:08:00.000Z', value: 27.05735206604004 },
  { timestamp: '2022-04-07T01:09:00.000Z', value: 27.816177368164062 },
  { timestamp: '2022-04-07T01:10:00.000Z', value: 28.398529052734375 },
  { timestamp: '2022-04-07T01:11:00.000Z', value: 28.804410934448242 },
  { timestamp: '2022-04-07T01:12:00.000Z', value: 29.351470947265625 },
  { timestamp: '2022-04-07T01:13:00.000Z', value: 29.598529815673828 },
  { timestamp: '2022-04-07T01:14:00.000Z', value: 29.84558868408203 },
  { timestamp: '2022-04-07T01:15:00.000Z', value: 30.233823776245117 },
  { timestamp: '2022-04-07T01:16:00.000Z', value: 30.48088264465332 },
  { timestamp: '2022-04-07T01:17:00.000Z', value: 30.674999237060547 },
  { timestamp: '2022-04-07T01:18:00.000Z', value: 30.851470947265625 },
  { timestamp: '2022-04-07T01:19:00.000Z', value: 31.01029396057129 },
  { timestamp: '2022-04-07T01:20:00.000Z', value: 31.1867637634277

In [38]:
let lastItem = newpoints[newpoints.length - 1]
//let entry = { timestamp: rounded, value: data.temperature } as Entry;
//points.push(entry);

In [39]:
lastItem

{ timestamp: '2022-04-07T01:21:00.000Z', value: 31.23970603942871 }


In [40]:
//lastItem.timestamp.setTime(lastItem.timestamp.getTime() + 1000 * 60);

In [41]:
lastItem.value = lastItem.value + 100;

131.2397060394287


In [42]:
newpoints

[
  { timestamp: '2022-04-07T01:07:00.000Z', value: 26.192646026611328 },
  { timestamp: '2022-04-07T01:08:00.000Z', value: 27.05735206604004 },
  { timestamp: '2022-04-07T01:09:00.000Z', value: 27.816177368164062 },
  { timestamp: '2022-04-07T01:10:00.000Z', value: 28.398529052734375 },
  { timestamp: '2022-04-07T01:11:00.000Z', value: 28.804410934448242 },
  { timestamp: '2022-04-07T01:12:00.000Z', value: 29.351470947265625 },
  { timestamp: '2022-04-07T01:13:00.000Z', value: 29.598529815673828 },
  { timestamp: '2022-04-07T01:14:00.000Z', value: 29.84558868408203 },
  { timestamp: '2022-04-07T01:15:00.000Z', value: 30.233823776245117 },
  { timestamp: '2022-04-07T01:16:00.000Z', value: 30.48088264465332 },
  { timestamp: '2022-04-07T01:17:00.000Z', value: 30.674999237060547 },
  { timestamp: '2022-04-07T01:18:00.000Z', value: 30.851470947265625 },
  { timestamp: '2022-04-07T01:19:00.000Z', value: 31.01029396057129 },
  { timestamp: '2022-04-07T01:20:00.000Z', value: 31.1867637634277

In [43]:
const O = [0, 0, 0];
const X = [255, 0, 0];

const cross = [
	X, O, O, O, O, O, O, X,
	O, X, O, O, O, O, X, O,
	O, O, X, O, O, X, O, O,
	O, O, O, X, X, O, O, O,
	O, O, O, X, X, O, O, O,
	O, O, X, O, O, X, O, O,
	O, X, O, O, O, O, X, O,
	X, O, O, O, O, O, O, X,
];


In [44]:
let result = await lastDetection(newpoints);
if (result)
{
    matrix.setPixels(cross);
}

Latest point anomaly detection:
The latest point, in row 15, is detected as an anomaly.


In [45]:
matrix.clear([0, 0, 0]);